In [55]:

pip install vaderSentiment


Note: you may need to restart the kernel to use updated packages.Installing collected packages: vaderSentiment



In [28]:
#import section

import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from  wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
cf.go_offline();
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")
pd.set_option('display.max_columns',None)
%matplotlib inline






In [29]:
df = pd.read_csv("amazon.csv")

In [30]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [37]:
# SORTING VALUES BY WILSON LOWER BOUND IN DESCENDING ORDER

In [35]:
df = df.sort_values("wilson_lower_bound", ascending= False)
df.drop("Unnamed: 0",inplace= True,axis= 1)

In [36]:
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [39]:
#FINDING THE MISSING VALUES

def missing_value_analysis(df):
    na_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
    na_miss = df[na_columns].isnull().sum().sort_values(ascending = True)
    ratio = (df[na_columns].isnull().sum()/df.shape[0]*100).sort_values(ascending = True)
    missing_df = pd.concat([na_miss,np.round(ratio,2)],axis=1,keys=["Missing Values",'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

In [40]:
missing_value_analysis(df)

,Missing Values,Ratio
reviewerName,1,0.02
reviewText,1,0.02


In [43]:
def check_dataFrame(df , heads =5 ,tails =5):
    print("Shape".center(82,'~'))
    print('Rows: {}',format (df.shape[0]))
    print('Columns:{}'.format(df.shape[1]))
    print(df.dtypes)
    print("".center(82,'~'))
    print(missing_value_analysis(df))
    print("Duplicated Values".center(82,'~'))
    print(df.duplicated().sum())
    print("Quantiles".center(82,'~'))
    print(df.quantile([0,0.05,0.50,0.95,1]).T)


check_dataFrame(df)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Shape~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Rows: {} 4915
Columns:11
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
              Missing Values  Ratio
reviewerName               1   0.02
reviewText                 1   0.02
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Duplicated Values~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Quantiles~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       0.00  0.05   0.50        0.95         1.00
overall                 1.0   2.0    5.0    5.000000     5.000000
day_diff                1.0  98.0  

In [44]:
#checking for unique values

def check_unique_val(dataframe):
    unique_df = pd.DataFrame({'Feature Names':dataframe.columns,
                                'Unique Values':[dataframe[i].nunique()\
                                    for i in dataframe.columns]})
    unique_df = unique_df.sort_values('Unique Values',ascending = False)
    unique_df = unique_df.reset_index(drop = True)
    return unique_df

check_unique_val(df)

,Feature Names,Unique Values
0,reviewText,4912
1,reviewerName,4594
2,reviewTime,690
3,day_diff,690
4,wilson_lower_bound,40
5,score_average_rating,28
6,score_pos_neg_diff,27
7,total_vote,26
8,helpful_yes,23
9,helpful_no,17


In [45]:
#cleaning data

df.reviewText.head()



2031    [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...
3449    I have tested dozens of SDHC and micro-SDHC ca...
4212    NOTE:  please read the last update (scroll to ...
317     If your card gets hot enough to be painful, it...
4672    Sandisk announcement of the first 128GB micro ...
Name: reviewText, dtype: object

In [46]:
#checking for a particular line

review_example = df.reviewText[2031]
review_example

'[[ UPDATE - 6/19/2014 ]]So my lovely wife bought me a Samsung Galaxy Tab 4 for Father\'s Day and I\'ve been loving it ever since.  Just as other with Samsung products, the Galaxy Tab 4 has the ability to add a microSD card to expand the memory on the device.  Since it\'s been over a year, I decided to do some more research to see if SanDisk offered anything new.  As of 6/19/2014, their product lineup for microSD cards from worst to best (performance-wise) are the as follows:SanDiskSanDisk UltraSanDisk Ultra PLUSSanDisk ExtremeSanDisk Extreme PLUSSanDisk Extreme PRONow, the difference between all of these cards are simply the speed in which you can read/write data to the card.  Yes, the published rating of most all these cards (except the SanDisk regular) are Class 10/UHS-I but that\'s just a rating... Actual real world performance does get better with each model, but with faster cards come more expensive prices.  Since Amazon doesn\'t carry the Ultra PLUS model of microSD card, I had 

In [74]:
#creating plots

constraints = ['#B34D022','#EBE00C','#1FEB00C','#0C92EB','#EB0CD5']
def categorical_varibale_summary(df,column_name):
    fig = make_subplots(rows=1,cols=2,
                            subplot_titles=("Countplot","Percentage"),
                            specs=[[{"types":"xy"},{'types':'domain'}]])
    fig.add_trace(go.Bar(y=df[column_name].value_counts().values.tolist(),
                        x=[str(i) for i in df[column_name].value_counts().index],
                        text = df[column_name].value_counts().values.tolist(),
                        textfont=dict(size=14),
                        name= column_name,
                        textposition ='auto',
                        showlegend= False,
                        marker= dict(color= constraints,
                        line = dict(color='#DBE6EC',
                        width = 1))),
                        row= 1,col =1)

    fig.add_trace(go.Pie(labels= df[column_name].value_counts().keys(),
                    values= df[column_name].value_counts().values,
                   textfont= dict(size = 18),
                   textposition ='auto',
                   showlegend= False,
                   name = column_name,
                   marker= dict(colors = constraints)),
                   row= 1 ,col= 2)


    fig.update_layout(title = {'text':column_name,
                            'y':0.9,
                            'x':0.5,
                            'xanchor':'center',
                            'yanchor':'top'},
                            template ='plotly_white')
    iplot(fig)

In [75]:
categorical_varibale_summary(df,'overall')

ValueError: 
Invalid key specified in an element of the 'specs' argument to make_subplots: 'types'
    Valid keys include: ['type', 'secondary_y', 'colspan', 'rowspan', 'l', 'r', 'b', 't']

In [48]:
#cleaning the punctuatin using regular expression

review_example = re.sub("[^a-zA-Z]",'',review_example)
review_example

'UPDATESomylovelywifeboughtmeaSamsungGalaxyTabforFathersDayandIvebeenlovingiteversinceJustasotherwithSamsungproductstheGalaxyTabhastheabilitytoaddamicroSDcardtoexpandthememoryonthedeviceSinceitsbeenoverayearIdecidedtodosomemoreresearchtoseeifSanDiskofferedanythingnewAsoftheirproductlineupformicroSDcardsfromworsttobestperformancewisearetheasfollowsSanDiskSanDiskUltraSanDiskUltraPLUSSanDiskExtremeSanDiskExtremePLUSSanDiskExtremePRONowthedifferencebetweenallofthesecardsaresimplythespeedinwhichyoucanreadwritedatatothecardYesthepublishedratingofmostallthesecardsexcepttheSanDiskregularareClassUHSIbutthatsjustaratingActualrealworldperformancedoesgetbetterwitheachmodelbutwithfastercardscomemoreexpensivepricesSinceAmazondoesntcarrytheUltraPLUSmodelofmicroSDcardIhadtododirectcomparisonsbetweentheSanDiskUltraExtremeandExtremePLUSAsmentionedinmyearlierreviewIpurchasedtheSanDiskUltraformyGalaxySMyquestionwasdidIwanttopayovermoreforacardthatisfasterthantheoneIalreadyownedOrIcouldpayalmostdoubletoget

In [ ]:
#converting all the texts to lower case




In [53]:
rt = lambda x:re.sub("[^a-zA-Z]", '',str(x))
df["reviewText"]= df["reviewText"].map(rt)
df["reviewText"]= df["reviewText"].str.lower()
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,updatesomylovelywifeboughtmeasamsunggalaxytabf...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,ihavetesteddozensofsdhcandmicrosdhccardsonedis...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,notepleasereadthelastupdatescrolltothebottomim...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,ifyourcardgetshotenoughtobepainfulitisdefectiv...,09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,sandiskannouncementofthefirstgbmicrosdtookinte...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [59]:
# Sentimental Analysis

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
df[['polarity','subjectivity']] = df ['reviewText'].apply(lambda Text:pd.Series(TextBlob(Text).sentiment))

for index , row in df['reviewText'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)

    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    if neg>pos:
        df.loc[index,'sentiment']= 'Negative'
    elif pos > neg:
        df.loc[index,'sentiment']= 'Postive'
    else:
        df.loc[index,'sentiment']= 'Neutral'


In [78]:
df[df["sentiment"]== "Positive"].sort_values("wilson_lower_bound",ascending= False).head(5)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment


2031    Neutral
3449    Neutral
4212    Neutral
317     Neutral
4672    Neutral
         ...   
4721    Neutral
4720    Neutral
4737    Neutral
4738    Neutral
4914    Neutral
Name: sentiment, Length: 4915, dtype: object

In [79]:
categorical_varibale_summary(df,'sentiment')

ValueError: 
Invalid key specified in an element of the 'specs' argument to make_subplots: 'types'
    Valid keys include: ['type', 'secondary_y', 'colspan', 'rowspan', 'l', 'r', 'b', 't']